In [17]:
import numpy as np
import astropy.io.fits as pf
import math
import matplotlib.pyplot as plt
from astropy.wcs import WCS

In [50]:

def find(cat, name, filtr):
    
    with open(cat, 'r') as f:
        with open(name, 'w') as y:
            lines = f.readlines()
            
            for i, line in enumerate(lines):
                if i<=2:
                    y.write(line) 
                for j in range(len(filtr)):
                    if i == filtr[j]+2:
                        y.write(line)     
                        

#cat = 'goodss_3dhst.v4.1.cat'
#name = 'c.txt'
#n = np.array([3,4])
#filtr = n
#find('goodss_3dhst.v4.1.cat','c.txt', n)




In [16]:
# Загружаем каталог из статьи
f = pf.open('asu.fit')
h = f[1].header
kat = f[1].data


ds  = kat['dset']
#print(kat['_RAJ2000'])
#print(kat['RAJ2000'])

fi = (ds == 3)

# Возьмём нужные, не знаю почему именно такие
Ra  = kat['_RAJ2000'][fi]
Dec = kat['_DEJ2000'][fi]
zphot = kat['zphot'][fi]
mag = kat['F160W'][fi]


print(Ra)

print(len(kat['_RAJ2000']))
print(len(kat['_DEJ2000']))
# А вот с представвлением wcs тут тяжко

w = WCS('asu.fit')
#w.printwcs()




[ 53.27737045  53.26937866  53.27024841  53.25611877  53.26578903
  53.29058075  53.15279007  53.15504074  53.14796066  53.13933182
  53.13800049  53.17950058  53.16550064]
50
50


In [15]:
# Проверяем работает ли перевод координат в пиксели

ff = pf.open('goodsn_3dhst.v4.0.F160W_orig_sci.fits')
hh = ff[0].header
#katt = ff[1].data


w = WCS('goodsn_3dhst.v4.0.F160W_orig_sci.fits')
print(w)
x = 189.228621
y = 62.238572
temp = w.all_world2pix(x, y, 1)  #  1-based (Fortran-like) coordinates  wh
print(temp[0])
print(temp[1])

i = 189.21
j = 62.23
te = w.all_world2pix(i, j, 1)  #  1-based (Fortran-like) coordinates  wh
print(te[0])
print(te[1])


WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN'  'DEC--TAN'  
CRVAL : 189.228621  62.238571999999998  
CRPIX : 10340.5  10240.5  
CD1_1 CD1_2  : -1.6666669999999999e-05  0.0  
CD2_1 CD2_2  : 0.0  1.6666669999999999e-05  
NAXIS    : 20480 20480
10340.5
10240.499999999789
10861.057488594512
9726.254946387193


In [49]:
# Загружаем данные с красными смещениями
data = np.genfromtxt('goodss_3dhst.v4.1.zout', skip_header=2)

z_spec = data[:,1]
z_a  = data[:,2]
numb = data[:,0]


# Запишем условие на обьекты
filt = (z_a >= 7)

# Отберём нужные
z_a1 = z_a[filt]
z_spec1 = z_spec[filt]
numb1 = numb[filt]
print(numb)

[  1.00000000e+00   2.00000000e+00   3.00000000e+00 ...,   5.05050000e+04
   5.05060000e+04   5.05070000e+04]


In [22]:
# Загружаем координаты обьектов из главного каталога
XX = np.genfromtxt('goodss_3dhst.v4.1.cat')[:, 1]
YY = np.genfromtxt('goodss_3dhst.v4.1.cat')[:, 2]


# Отберём нужные
X1 = XX[filt]
Y1 = YY[filt]



# Первая проверка
print(len(X1), len(z_a1))



1102 1102
[   304.    386.    569. ...,  50445.  50449.  50504.]


In [26]:
# Из выбранных обьектов отбираем те, которые находятся в нужной области и формируем новые массивы координат, 
# одновременно с новыми массивами для красного смещения и номерами обьектов
X0 = []
Y0 = []
rs = []
n =  []
for i in range (len(X1)):

    if (0 < X1[i] - 6000 < 4000) and (0 < Y1[i] - 10000 < 4000):
        X0.append(X1[i] - 6000)
        Y0.append(Y1[i] - 10000)
        rs.append(z_a1[i])
        n.append(numb1[i])
        
# Cоздадим таблицу всех обьектов в данной области у которых z>=7
Yes0 = np.vstack((X0, Y0))
np.savetxt('cat.dat', np.transpose(Yes0), delimiter="  ", fmt="%s")


323


In [6]:
# Вторая проверка
print(len(rs),len(X0),len(Y0),len(n))

323 323 323 323


In [7]:
# Третья проверка 
k = np.array([3.99,4.222,5.99,6.000])
As = np.zeros((len(k),len(k)))
tk = np.zeros(len(k))


for j in range (len(k)):   
     tk = abs(k - k[j])
     As[j,:] = tk
        
print(As)

[[ 0.     0.232  2.     2.01 ]
 [ 0.232  0.     1.768  1.778]
 [ 2.     1.768  0.     0.01 ]
 [ 2.01   1.778  0.01   0.   ]]


In [28]:
# Создаём матрицы расстояний для координат и красного смещения
Ax = np.zeros((len(X0),len(X0)))
Ay = np.zeros((len(Y0),len(Y0)))
Az = np.zeros((len(rs),len(rs)))

tx = np.zeros(len(X0))
ty = np.zeros(len(Y0))
tz = np.zeros(len(rs))
for j in range (len(X0)):   
     tx = abs(X0 - X0[j])
     Ax[j,:] = tx
     ty = abs(Y0 - Y0[j])
     Ay[j,:] = ty
     tz = abs(rs - rs[j])
     Az[j,:] = tz
  

In [23]:
# Четвёртая проверка
print('X')
print(Ax)






X
[[    0.     1105.658  1050.749 ...,   639.545   585.607  1060.237]
 [ 1105.658     0.       54.909 ...,  1745.203   520.051  2165.895]
 [ 1050.749    54.909     0.    ...,  1690.294   465.142  2110.986]
 ..., 
 [  639.545  1745.203  1690.294 ...,     0.     1225.152   420.692]
 [  585.607   520.051   465.142 ...,  1225.152     0.     1645.844]
 [ 1060.237  2165.895  2110.986 ...,   420.692  1645.844     0.   ]]


In [10]:
# Смотрим сколько обьектов находятся в пределах 2.5 пикселя от 15-го. 
G = []
G = Ax[15,:]
k = 0
for i in range (len(G)):
    if (0 < G[i] < 2.5):
        k+=1
print(k)        

0


In [44]:
# Ищем сколько в каждой строке обьектов, которые находятся близко друг другу и
# имеют одинаковые красные смещения

candidatsx = [np.sum((Ax[i] > 0) & (Ax[i] < 3)) for i in range(len(X0))]
print(candidatsx)
print()

candidatsy = [np.sum((Ay[i] > 0) & (Ay[i] < 3)) for i in range(len(Y0))]
print(candidatsy)
print()


# Во избежании тех случаев, когда разница берётся от одного обьекта
for i in range (len(Az)):
    for j in range (len(Az)):
        if i == j :
            Az[i,j] = 1000
            
        
candidatsz = [np.sum((Az[i] >= 0) & (Az[i]  < 0.00000000001)) for i in range(len(rs))]
print(Az)


[0, 0, 0, 1, 3, 1, 0, 2, 1, 1, 1, 1, 1, 2, 2, 0, 0, 1, 0, 0, 0, 0, 2, 0, 1, 0, 2, 1, 1, 1, 0, 2, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 2, 0, 2, 1, 1, 2, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 3, 3, 1, 0, 4, 2, 2, 0, 2, 0, 0, 0, 2, 1, 0, 0, 1, 0, 1, 1, 0, 1, 2, 0, 3, 0, 1, 0, 3, 0, 2, 0, 1, 1, 1, 1, 2, 3, 3, 0, 0, 0, 3, 1, 0, 1, 0, 1, 2, 0, 2, 0, 0, 1, 0, 1, 1, 2, 2, 0, 1, 0, 2, 0, 1, 2, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 2, 1, 1, 0, 0, 0, 2, 0, 2, 1, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 3, 1, 0, 0, 2, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 3, 0, 0, 0, 2, 2, 1, 0, 0, 1, 0, 0, 0, 1, 2, 3, 2, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 0, 3, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 4, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 2, 2, 1, 0, 1, 0, 3, 0, 1, 0, 3, 1, 1, 0, 0, 3, 0, 0, 0, 1, 0, 2, 0, 4, 1, 0, 2, 1, 3, 1, 2, 0, 0, 2, 0, 1, 4, 1, 2, 0, 2, 0, 0, 1, 1, 3, 0, 1, 2, 1, 2, 2, 0, 1, 0, 0, 2, 1, 2, 0, 1, 0, 0, 2, 1, 2, 0, 0, 0, 1, 0, 3, 1, 0, 1, 4, 1, 2, 1, 1, 1]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [46]:
# Из столбцов координат и красного смещения выбираем обьекты, где выполняются все три критерия
X0 = np.array(X0)
x = X0[np.where((np.array(candidatsx)>0) & (np.array(candidatsy)>0)  & (np.array(candidatsz)>0) )[0]]
print()

Y0 = np.array(Y0)
y = Y0[np.where( (np.array(candidatsx)>0)  & (np.array(candidatsy)>0)  & (np.array(candidatsz)>0) )[0]]


rs = np.array(rs)
z = rs[np.where((np.array(candidatsx)>0) & (np.array(candidatsy)>0)  & (np.array(candidatsz)>0) )[0]]


n = np.array(n)
print(len(n))
m = n[np.where((np.array(candidatsx)>0) & (np.array(candidatsy)>0)  & (np.array(candidatsz)>0) )[0]]

print(m)


323
[ 23480.  23750.  23755.  23893.  23920.  23951.  24511.  24689.  24692.
  25014.  25097.  25111.  25264.  25274.  25395.  25537.  25546.  25608.
  25613.  25704.  25888.  25938.  26010.  26052.  26080.  26460.  26640.
  26711.  26719.  26754.  26775.  26823.  26940.  26946.  27030.  27123.
  27215.  27239.  27288.  27359.  27365.  27828.  27905.  27921.  28025.
  28081.  28089.  28099.  28174.  28179.  28185.  28242.  28276.  28280.
  28283.  28298.  28489.  28513.  28526.  28924.  28933.  28937.  29099.
  29161.  29184.  29206.  29391.  29398.  29589.  29651.  29659.  30065.
  30511.  30543.  30885.  30888.  31223.  31496.  31500.  31504.  31540.
  32081.  32405.]


In [51]:
find('goodss_3dhst.v4.1.cat','c_nw.txt', m)

In [47]:
# Пятая проверка 
A = np.array([[1.3,2,3,8,9],[4,5,6,8,9],[1,1,1,9,9],[1,9,9,9,7],[3,2,3,4,5]])
candidats1 = [np.sum((A[i]>0) & (A[i] < 2)) for i in range(5)]
print(candidats1)
print()

B = np.array([[1.8,2,3,6,6],[4,5,6,8,6],[4,4,4,9,1],[1,0,9,9,3],[3,3,3,3,3]])
candidats2 = [np.sum((B[i]>0) & (B[i] < 2)) for i in range(5)]
print(candidats2)
print()

C = np.array([[1,2,3,1,4],[4,5,6,8,4],[1,1,1,9,4],[1.0,9,9,9.0,4],[1,4,4,4,4]])
candidats3 = [np.sum((C[i]>0) & (C[i] < 2)) for i in range(5)]
print(candidats3)
print()

g=np.array([11.4,21,31,41,91])
g[np.where((np.array(candidats1)>0) & (np.array(candidats2)>0)  & (np.array(candidats3)>0) )[0]]


[1, 0, 3, 1, 0]

[1, 0, 1, 1, 0]

[2, 0, 3, 1, 1]



array([ 11.4,  31. ,  41. ])

In [14]:
# Составляем таблицу парных обьектов
Yes = np.vstack((m, x, y, z))
np.savetxt('intr.dat', np.transpose(Yes), delimiter="  ", fmt="%s")

In [15]:
dat = np.genfromtxt('intr.dat')

x = dat[:,1]
y = dat[:,2]
z = dat[:,3]

In [16]:
# Смотрим что получилось
print(x)
print()
print(y)
print()
print(z)



[ 2383.263  2572.512  2319.53   2683.477  2717.717  2865.121  2610.95
  1772.009  2262.227  1911.071  3181.561  2074.046  2943.064  3125.356
  2907.912  3269.101  2973.986  1986.187  2107.084  2233.588  1996.622
  3003.382  1413.702  2895.112  1846.888  3127.1    2332.784  2906.459
  1864.962  2976.02   1671.041  2506.939  3003.144  2836.245  1717.926
  2266.242  1715.274  3271.379  2060.298  1436.512  2099.998  2921.756
  2146.851  2754.042  2507.923  2038.893  1188.653  2907.183  3036.838
  2570.138  1412.206  2071.858  2895.537  2719.517  1717.408  2233.779
  1997.566  1866.281  1673.201  2037.43   2568.77   2332.738  1914.999
  2343.784  2511.504  2599.857  1867.999  2046.732  2263.464  1997.029]

[  909.132   980.75    981.609  1013.105  1012.93   1030.942  1184.147
  1226.97   1226.557  1309.553  1325.229  1369.653  1368.801  1403.655
  1446.813  1448.609  1464.281  1466.106  1486.186  1536.462  1547.216
  1563.61   1568.588  1565.354  1710.908  1727.284  1728.076  1728.69
  1738